# 6. 部署

在前面几节中，我们抽象出了一些公有代码，放在 `util.py` 中。我们复用这些代码，快速训练一个 LightGBM 模型

In [1]:
DIRECTORY = './data'
TRAIN_FILE = 'adult/adult.data'
TEST_FILE = 'adult/adult.test'
MODEL_FILE = 'model_best.txt'
KKV_FILE = 'label_encoder_deploy.json'
COLS = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
        'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
        'hours-per-week', 'native-country', 'income']
LABEL_COL = 'income'

In [2]:
# coding: utf-8
import numpy as np
import pandas as pd
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from swifter import swifter

import util

In [3]:
# 明确指定数据类型是好文明
DTYPE = {
    'age': np.int64,
    'workclass': np.dtype('object'),
    'fnlwgt': np.int64,
    'education': np.dtype('object'),
    'education-num': np.int64,
    'marital-status': np.dtype('object'),
    'occupation': np.dtype('object'),
    'relationship': np.dtype('object'),
    'race': np.dtype('object'),
    'sex': np.dtype('object'),
    'capital-gain': np.int64,
    'capital-loss': np.int64,
    'hours-per-week': np.int64,
    'native-country': np.dtype('object'),
    'income': np.dtype('object')
}

## 一、模型训练

我们用前面学到的技巧，训练一版模型

In [4]:
# 读入 CSV 文件
csv_file = util.gen_abspath(DIRECTORY, TRAIN_FILE)
df = util.read_csv(csv_file, sep=',', header=None, dtype=DTYPE)
df.columns=COLS

# 特征与标号
X = df.drop(LABEL_COL, axis=1)  # features
y = df[LABEL_COL].apply(lambda e: 0 if e == ' <=50K' else 1)  # label

# 处理类别特征
X, kkv, cat_feats = util.label_encoder_trim(X)

# 保存 encoder 的映射关系
kkv_path = util.gen_abspath(DIRECTORY, KKV_FILE)
util.save_label_encoder(kkv, kkv_path)

Pandas Apply:   0%|          | 0/32561 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/32561 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/32561 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/32561 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/32561 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/32561 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/32561 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/32561 [00:00<?, ?it/s]

In [5]:
# 分割数据集
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)
lgb_train = lgb.Dataset(X_train,
                        y_train,
                        categorical_feature=cat_feats, 
                        free_raw_data=True)
lgb_eval = lgb.Dataset(X_test,
                    y_test,
                    reference=lgb_train,
                    categorical_feature=cat_feats, 
                    free_raw_data=True)

# 配置训练参数：使用在 Adult 数据集上优化过的超参数
best_params = {
    "objective": "binary",
    "boosting_type": "gbdt",
    "metric": 'auc',
    "num_leaves": 34,
    "learning_rate": 0.08,
    "feature_fraction": 0.4,
    "bagging_fraction": 0.9,
    "bagging_freq": 5,
    "subsample": 0.9,
    "verbose": 1
}

# 处理样本数据倾斜
positive_ratio = sum(y_train) / len(y_train)
tolerance = 0.1
if positive_ratio > 0.5 + tolerance or positive_ratio < 0.5 - tolerance:
    # if dataset is highly imbalanced
    weight = util.gen_scale_pos_weight(y_train)
    best_params["scale_pos_weight"] = weight
    print(f"Warning: Sample imbalance, set scale_pos_weight={weight:.3f}")

In [6]:
# 训练
gbm = lgb.train(best_params,
                lgb_train,
                num_boost_round=300,
                valid_sets=lgb_eval,
                callbacks=[lgb.early_stopping(stopping_rounds=30),
                           lgb.log_evaluation(10),
                           util.AdaptiveLearningRate(learning_rate=0.1, decay_rate=0.9, patience=10).callback],
                categorical_feature=cat_feats
)

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Info] Number of positive: 6270, number of negative: 19778
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000542 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 695
[LightGBM] [Info] Number of data points in the train set: 26048, number of used features: 14
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240709 -> initscore=-1.148794
[LightGBM] [Info] Start training from score -1.148794
Training until validation scores don't improve for 30 rounds
Learning rate ==> 0.090 (-0.0100

In [7]:
# 评估模型
y_pred = gbm.predict(X_test)
y_label, threshold = util.eval_binary(y_true=y_test, y_pred=y_pred, ret=True)

threshold: 0.70658
accuracy: 0.88070
precision: 0.76361
recall: 0.73202
f1_score: 0.74748
auc: 0.93199
cross-entropy loss: 0.32999
True Positive (TP): 1150
True Negative (TN): 4586
False Positive (FP): 356
False Negative (FN): 421
confusion matrix:
[[4586  356]
 [ 421 1150]]


In [8]:
# 保存模型
model_path = util.gen_abspath(DIRECTORY, MODEL_FILE)
gbm.save_model(model_path)

## 二、模型部署

In [9]:
# 从模型文件加载模型
bst = lgb.Booster(model_file=model_path)

### 2.1 离线部署

如果这时候有一行数据进来，怎么对它进行推理呢？

In [10]:
# 假设这行数据长这样
csv_file = util.gen_abspath(DIRECTORY, TEST_FILE)
df_test = util.read_csv(csv_file, sep=',', header=None)
df_test.columns=COLS

df_one = df_test.iloc[3:4,:]
df_one

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.


首先对它进行预处理

In [11]:
# 转成特征与标号
XX = df_one.drop(LABEL_COL, axis=1)  # features
yy = df_one[LABEL_COL].apply(lambda e: 0 if e == ' <=50K.' else 1)  # label

# 处理类别特征
XX, _ = util.load_label_encoder(XX, kkv_path)

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
XX

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
3,44,4,160323,15,10,2,7,0,2,1,7688,0,40,39


再进行预测

In [13]:
# 输入可以是 DataFrame
y_guess = bst.predict(XX)[0]
y_label = 1 if y_guess > threshold else 0

print(f'y_guess: {y_guess:.5f}')
print(f'y_label: {y_label}')
print(f'y_true: {list(yy)[0]}')

y_guess: 0.98814
y_label: 1
y_true: 1


In [14]:
# 也可以是 Series
arr = XX.iloc[0]
y_guess = bst.predict([arr.astype(np.float32)])
y_guess

array([0.98814364])

In [15]:
# 也可以是 np.array
np_arr = np.array([XX.iloc[0]])
y_guess = bst.predict(np_arr)
y_guess

array([0.98814364])

### 2.2 在线部署

可以用 FastAPI 进行在线部署。注意现实中需要对字段进行校验，对有问题的数据设计特殊策略，比如 异常兜底、异常报警、空值填充 等

关于 FastAPI 的更多信息，可以查阅我的博客 [FastAPI 初见](https://www.luochang.ink/posts/fastapi/) 及配套代码 [calendar-api](https://github.com/luochang212/calendar-api)